# O2O商铺食品安全相关评论发现

https://www.datafountain.cn/competitions/370
    
## 赛题背景
互联网经济蓬勃发展的背景下,食品经营模式发生了天翻地覆的变化,人们的消费习惯也悄然发生了转变。通过点击手机APP上自己喜欢的食品,这些食品就能按时准确送达指定的区域，这就是当下最受学生和白领喜欢的外卖。然而随着其迅猛发展带来了一定的食品安全隐患，食品安全事故的发生对消费者、外卖平台、食品商家和社会的危害性远远超出想象。  
本赛题旨在通过对O2O店铺评论的监测，加强对店铺的食品安全监管。

## 赛题任务
本赛题提供了10000条对O2O店铺的评论文本训练数据，分为与食品安全有关和与食品安全无关两个类别。参赛者需要根据训练集构造文本分类模型，预测2000条测试集中的评论是否与食品安全有关。

## 分析
简单看了一下，就是文本2分类问题

# data explore

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)  # 设置显示数据的最大列数，防止出现省略号…，导致数据显示不全
pd.set_option('expand_frame_repr', False)  # 当列太多时不自动换行

In [2]:
df = pd.read_csv('data_origin/train.csv')

df.head()

In [4]:
df.sample(10)

,label comment
1962,0\t老顾客了。但是味道没有以前好了。才炸好得。吃起来很闷的感觉。
8527,0\t面条不错，蛮好吃，去过很多次，满是喜欢，下次还去
9623,0\t服务很好，会详细的介绍各类奶茶
660,0\t第二次来乐。 好。菜新鲜。种类多
2796,1\t辣翅臭了，不知道放了多久。
1155,1\t我想说里脊肉是有味道的，手抓饼也有臭味，这个怎么吃，差评，差得不能再差！
8543,0\t还不错，味道也不错，还有环境也不错
4708,0\t老顾客。好久没有光顾了
6453,0\t环境不错、服务也挺好、上菜也很快
1812,0\t环境不错，味道可以，满意，服务很不错哟


# fasttext

## train

In [6]:
df.columns

Index(['label\tcomment'], dtype='object')

In [9]:
df['label'] = df['label\tcomment'].str.split('\t').str.get(0)
df['comment'] = df['label\tcomment'].str.split('\t').str.get(1)

In [11]:
import jieba

In [12]:
def t(s):
    return ' '.join(jieba.lcut(s))
df['word_seg'] = df.comment.map(t)

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/7j/kgtjln3x2dj2g2v57d5vncyw0000gp/T/jieba.cache
Loading model cost 0.848 seconds.
Prefix dict has been built succesfully.


In [22]:
df['train'] = df.word_seg + ' __label__' + df.label + '\n'

In [23]:
df.head()

,label comment,label,comment,word_seg,train
0,0\t一如既往地好吃，希望可以开到其他城市,0,一如既往地好吃，希望可以开到其他城市,一如既往 地 好吃 ， 希望 可以 开 到 其他 城市,一如既往 地 好吃 ， 希望 可以 开 到 其他 城市 __label__0\n
1,0\t味道很不错，分量足，客人很多，满意,0,味道很不错，分量足，客人很多，满意,味道 很 不错 ， 分量 足 ， 客人 很多 ， 满意,味道 很 不错 ， 分量 足 ， 客人 很多 ， 满意 __label__0\n
2,0\t下雨天来的，没有想象中那么火爆。环境非常干净，古色古香的，我自己也是个做服务行业的，我...,0,下雨天来的，没有想象中那么火爆。环境非常干净，古色古香的，我自己也是个做服务行业的，我都觉得...,下雨天 来 的 ， 没有 想象 中 那么 火爆 。 环境 非常 干净 ， 古色古香 的 ， ...,下雨天 来 的 ， 没有 想象 中 那么 火爆 。 环境 非常 干净 ， 古色古香 的 ， ...
3,0\t真心不好吃 基本上没得好多味道,0,真心不好吃 基本上没得好多味道,真心 不 好吃 基本上 没 得 好多 味道,真心 不 好吃 基本上 没 得 好多 味道 __label__0\n
4,0\t少送一个牛肉汉堡 而且也不好吃 特别是鸡肉卷 **都不想评论了 谁买谁知道,0,少送一个牛肉汉堡 而且也不好吃 特别是鸡肉卷 **都不想评论了 谁买谁知道,少送 一个 牛肉 汉堡 而且 也 不 好吃 特别 是 鸡肉 卷 * * 都 不想...,少送 一个 牛肉 汉堡 而且 也 不 好吃 特别 是 鸡肉 卷 * * 都 不想...


In [24]:
with open('data_gen/train.txt', 'w', encoding='utf8') as f:
    f.writelines(df.train.tolist())

In [16]:
import fasttext

In [25]:
model = fasttext.train_supervised('data_gen/train.txt')

## val

In [26]:
df_test = pd.read_csv('data_origin/test_new.csv')

In [30]:
df_test['comment_seg'] = df_test.comment.map(t)

In [38]:
df_test.head()

,id,comment,comment_seg,result,label
0,0011f384-9e54-4fb4-a272-330a6cab6804,糯米团是我小时候的记忆了，吃起还是好吃，只是小时候的油条没有这么硬！油茶也还好！可以试试,糯米 团是 我 小时候 的 记忆 了 ， 吃 起 还是 好吃 ， 只是 小时候 的 油条 没...,"((__label__0,), [0.9305034875869751])",0
1,00223e4f-47e1-4fc8-9657-06444a7de9a5,满满的五星好评，口味好，服务好，特别喜欢，昨天第一次买，今天就回购了，买的刨奶，店长问我加腰...,满满的 五星 好评 ， 口味 好 ， 服务 好 ， 特别 喜欢 ， 昨天 第一次 买 ， 今...,"((__label__0,), [0.9706935286521912])",0
2,00225350-c169-435c-84cf-970068df5b12,好喝！经常会再去买来喝！就是排队的人太多了,好喝 ！ 经常 会 再 去 买来 喝 ！ 就是 排队 的 人太多 了,"((__label__0,), [0.9995092153549194])",0
3,00a3190c-90c1-44c3-b809-7a9b1314cd27,三个人订的四人餐，菜量大没吃完，问道不错。,三个 人订 的 四人餐 ， 菜量 大 没 吃 完 ， 问道 不错 。,"((__label__0,), [0.9895778298377991])",0
4,00b3f76e-fda3-42cd-8884-25e03a5dba64,好的一如既往，真真爱上了自助炒饭自助八宝粥自助冰粉！！！喜欢所有菜和肉，两女一男吃两份两人餐...,好 的 一如既往 ， 真真 爱上 了 自助 炒饭 自助 八宝粥 自助 冰粉 ！ ！ ！ 喜欢...,"((__label__0,), [0.994170606136322])",0


In [32]:
df_test['result'] = df_test.comment_seg.map(model.predict)

In [37]:
df_test['label'] = df_test.result.str.get(0).str.get(0).str.get(-1)

In [39]:
df_test[['id', 'label']].to_csv('data_gen/result_fasttext.csv', index=False)

## score

### 当前排名： 第 77 名 最高得分：0.86829270

# tf.keras

In [41]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

print(tf.__version__)

1.14.0


In [43]:
df = pd.read_csv('data_origin/train.csv')
df['label'] = df['label\tcomment'].str.split('\t').str.get(0)
df['comment'] = df['label\tcomment'].str.split('\t').str.get(1)
df.head()

,label comment,label,comment
0,0\t一如既往地好吃，希望可以开到其他城市,0,一如既往地好吃，希望可以开到其他城市
1,0\t味道很不错，分量足，客人很多，满意,0,味道很不错，分量足，客人很多，满意
2,0\t下雨天来的，没有想象中那么火爆。环境非常干净，古色古香的，我自己也是个做服务行业的，我...,0,下雨天来的，没有想象中那么火爆。环境非常干净，古色古香的，我自己也是个做服务行业的，我都觉得...
3,0\t真心不好吃 基本上没得好多味道,0,真心不好吃 基本上没得好多味道
4,0\t少送一个牛肉汉堡 而且也不好吃 特别是鸡肉卷 **都不想评论了 谁买谁知道,0,少送一个牛肉汉堡 而且也不好吃 特别是鸡肉卷 **都不想评论了 谁买谁知道


In [44]:
df.label.value_counts()

0    8489
1    1511
Name: label, dtype: int64

In [45]:
from collections import Counter

In [61]:
df['comment_seg'] = df.comment.map(jieba.lcut)

In [69]:
df.head()

,label comment,label,comment,comment_seg,comment_num
0,0\t一如既往地好吃，希望可以开到其他城市,0,一如既往地好吃，希望可以开到其他城市,"[一如既往, 地, 好吃, ，, 希望, 可以, 开, 到, 其他, 城市]","[71, 679, 15, 1, 193, 27, 719, 52, 156, 2778]"
1,0\t味道很不错，分量足，客人很多，满意,0,味道很不错，分量足，客人很多，满意,"[味道, 很, 不错, ，, 分量, 足, ，, 客人, 很多, ，, 满意]","[10, 5, 12, 1, 42, 53, 1, 680, 109, 1, 96]"
2,0\t下雨天来的，没有想象中那么火爆。环境非常干净，古色古香的，我自己也是个做服务行业的，我...,0,下雨天来的，没有想象中那么火爆。环境非常干净，古色古香的，我自己也是个做服务行业的，我都觉得...,"[下雨天, 来, 的, ，, 没有, 想象, 中, 那么, 火爆, 。, 环境, 非常, 干...","[2432, 22, 2, 1, 29, 984, 525, 199, 1996, 6, 2..."
3,0\t真心不好吃 基本上没得好多味道,0,真心不好吃 基本上没得好多味道,"[真心, 不, 好吃, , 基本上, 没, 得, 好多, 味道]","[201, 19, 15, 4, 883, 40, 72, 284, 10]"
4,0\t少送一个牛肉汉堡 而且也不好吃 特别是鸡肉卷 **都不想评论了 谁买谁知道,0,少送一个牛肉汉堡 而且也不好吃 特别是鸡肉卷 **都不想评论了 谁买谁知道,"[少送, 一个, 牛肉, 汉堡, , 而且, 也, 不, 好吃, , 特别, 是, 鸡肉...","[1997, 58, 143, 633, 4, 62, 11, 19, 15, 4, 49,..."


In [133]:
def buildDataset(words, vocabulary_size):
    """

    :param words: 所有文章分词后的一个words list
    :param vocabulary_size: 取频率最高的词数
    :return:
        data 编号列表，编号形式
        count 前50000个出现次数最多的词
        dictionary 词对应编号
        reverse_dictionary 编号对应词
    """
    count = [['<PAD>', -1], ['<UNK>', -1]]
    # 前50000个出现次数最多的词
    count.extend(Counter(words).most_common(vocabulary_size - 2))
    # 生成 dictionary，词对应编号, word:id(0-49999)
    # 词频越高编号越小
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    # data把数据集的词都编号
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 1  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    # 记录UNK词的数量
    count[0][1] = unk_count
    # 编号对应词的字典
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reverse_dictionary

In [134]:
data, count, dictionary, reverse_dictionary = buildDataset([ii for i in df.comment.map(jieba.lcut).tolist() for ii in i], vocab_size)

In [135]:
def t1(s):
    return [dictionary[i] if i in dictionary else 0 for i in s]

In [136]:
df['comment_num'] = df.comment_seg.map(t1)

In [137]:
train_data = df['comment_num'].values

In [138]:
len(train_data[0])

10

In [139]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=dictionary["<PAD>"],
                                                        padding='post',
                                                        maxlen=64)

# test_data = keras.preprocessing.sequence.pad_sequences(test_data,
#                                                        value=dictionary["<PAD>"],
#                                                        padding='post',
#                                                        maxlen=256)

In [140]:
len(train_data[0])

64

In [141]:
# input shape is the vocabulary count used for the movie reviews (10,000 words)
vocab_size = 5000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 64))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 64)          320000    
_________________________________________________________________
global_average_pooling1d_4 ( (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 16)                1040      
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 17        
Total params: 321,057
Trainable params: 321,057
Non-trainable params: 0
_________________________________________________________________


In [142]:
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [143]:
train_labels = df.label.values

In [144]:
x_val = train_data[:2000]
partial_x_train = train_data[2000:]

y_val = train_labels[:2000]
partial_y_train = train_labels[2000:]

In [145]:
partial_x_train

array([[3000,   15,  120, ...,    0,    0,    0],
       [  51,    9,    2, ...,    0,    0,    0],
       [  26,   13,    2, ...,    0,    0,    0],
       ...,
       [ 402,   64,  430, ...,    0,    0,    0],
       [  26,   11,    6, ...,    0,    0,    0],
       [ 172, 1230,    2, ...,    0,    0,    0]], dtype=int32)

In [146]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=200,
                    validation_data=(x_val, y_val),
                    verbose=1)

Train on 8000 samples, validate on 2000 samples
Epoch 1/40
8000/8000 [==============================] - 0s 43us/sample - loss: 0.5871 - acc: 0.8475 - val_loss: 0.4763 - val_acc: 0.8545
Epoch 2/40
8000/8000 [==============================] - 0s 20us/sample - loss: 0.4244 - acc: 0.8475 - val_loss: 0.3824 - val_acc: 0.8545
Epoch 3/40
8000/8000 [==============================] - 0s 21us/sample - loss: 0.3831 - acc: 0.8475 - val_loss: 0.3609 - val_acc: 0.8545
Epoch 4/40
8000/8000 [==============================] - 0s 21us/sample - loss: 0.3579 - acc: 0.8475 - val_loss: 0.3351 - val_acc: 0.8545
Epoch 5/40
8000/8000 [==============================] - 0s 21us/sample - loss: 0.3242 - acc: 0.8539 - val_loss: 0.2991 - val_acc: 0.8655
Epoch 6/40
8000/8000 [==============================] - 0s 21us/sample - loss: 0.2793 - acc: 0.8685 - val_loss: 0.2570 - val_acc: 0.8730
Epoch 7/40
8000/8000 [==============================] - 0s 21us/sample - loss: 0.2333 - acc: 0.8915 - val_loss: 0.2158 - val_acc: 

## val

In [147]:
df_test = pd.read_csv('data_origin/test_new.csv')

In [149]:
df_test['comment_seg'] = df_test.comment.map(jieba.lcut)
df_test['comment_num'] = df_test.comment_seg.map(t1)

In [150]:
test_data = df_test.comment_num.values

In [152]:
test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=dictionary["<PAD>"],
                                                       padding='post',
                                                       maxlen=64)

In [155]:
df_test['prob'] = model.predict(test_data).reshape(-1)

In [156]:
df_test.head()

,id,comment,comment_seg,comment_num,prob
0,0011f384-9e54-4fb4-a272-330a6cab6804,糯米团是我小时候的记忆了，吃起还是好吃，只是小时候的油条没有这么硬！油茶也还好！可以试试,"[糯米, 团是, 我, 小时候, 的, 记忆, 了, ，, 吃, 起, 还是, 好吃, ，,...","[2116, 0, 17, 3356, 3, 4119, 4, 2, 10, 359, 34...",1.575291e-03
1,00223e4f-47e1-4fc8-9657-06444a7de9a5,满满的五星好评，口味好，服务好，特别喜欢，昨天第一次买，今天就回购了，买的刨奶，店长问我加腰...,"[满满的, 五星, 好评, ，, 口味, 好, ，, 服务, 好, ，, 特别, 喜欢, ，...","[3039, 1145, 205, 2, 78, 9, 2, 25, 9, 2, 50, 3...",8.940697e-08
2,00225350-c169-435c-84cf-970068df5b12,好喝！经常会再去买来喝！就是排队的人太多了,"[好喝, ！, 经常, 会, 再, 去, 买来, 喝, ！, 就是, 排队, 的, 人太多, 了]","[111, 8, 94, 115, 60, 21, 1559, 183, 8, 40, 28...",2.145767e-06
3,00a3190c-90c1-44c3-b809-7a9b1314cd27,三个人订的四人餐，菜量大没吃完，问道不错。,"[三个, 人订, 的, 四人餐, ，, 菜量, 大, 没, 吃, 完, ，, 问道, 不错, 。]","[258, 0, 3, 777, 2, 517, 216, 41, 10, 108, 2, ...",1.828074e-04
4,00b3f76e-fda3-42cd-8884-25e03a5dba64,好的一如既往，真真爱上了自助炒饭自助八宝粥自助冰粉！！！喜欢所有菜和肉，两女一男吃两份两人餐...,"[好, 的, 一如既往, ，, 真真, 爱上, 了, 自助, 炒饭, 自助, 八宝粥, 自助...","[9, 3, 72, 2, 0, 2456, 4, 426, 437, 426, 3000,...",1.788139e-07


In [158]:
df_test['label'] = np.where(df_test['prob']>0.5, 1, 0)

In [159]:
df_test.label.value_counts()

0    1713
1     287
Name: label, dtype: int64

In [160]:
df_test[['id', 'label']].to_csv('data_gen/result_tfkeras.csv', index=False)

## score

### 0.85579200000